In [2]:
import sys
sys.path.append("../")
from pyproj import CRS
import numpy as np
import sys
from osgeo import gdal
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from datetime import datetime
from time import time
from collections import Counter
import pandas as pd #for data analysing
import openpyxl
import os



In [3]:
##### pour savoir ou je suis dans mon environnement
os.getcwd()

'/home/charlottepoussin/variable'

In [4]:
NDSI_SON_beforSL_b = xr.open_dataset('NDSI_DJF_beforSL_b_nc.nc')
NDSI_SON_beforSL_a = xr.open_dataset('NDSI_DJF_beforSL_a_nc.nc') 

In [5]:
###j'ouvre mon dataset
summer_mask = xr.open_dataset('summer_mask_80.nc')
summer_mask

<xarray.Dataset>
Dimensions:    (longitude: 14234, latitude: 6200)
Coordinates:
  * longitude  (longitude) float64 5.7 5.7 5.701 5.701 ... 10.55 10.55 10.55
  * latitude   (latitude) float64 47.85 47.85 47.85 47.85 ... 45.74 45.74 45.74
Data variables:
    crs        int32 ...
    layer      (latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-03-25 10:39:14

In [6]:
NDSI_SON_beforSL = xr.concat([NDSI_SON_beforSL_a, NDSI_SON_beforSL_b], dim='longitude')

In [7]:
NDSI_SON_beforSL.z.values

array(['1984-12-01', '1985-01-01', '1985-02-01', '1985-12-01',
       '1986-01-01', '1986-02-01', '1986-12-01', '1987-02-01',
       '1987-12-01', '1988-01-01', '1988-02-01', '1988-12-01',
       '1989-01-01', '1989-02-01', '1989-12-01', '1990-01-01',
       '1990-02-01', '1990-12-01', '1991-01-01', '1991-02-01',
       '1991-12-01', '1992-01-01', '1992-02-01', '1992-12-01',
       '1993-01-01', '1993-02-01', '1993-12-01', '1994-01-01',
       '1994-02-01', '1994-12-01', '1995-01-01', '1995-02-01',
       '1995-12-01', '1996-01-01', '1996-02-01', '1996-12-01',
       '1997-01-01', '1997-02-01', '1997-12-01', '1998-01-01',
       '1998-02-01', '1998-12-01', '1999-01-01', '1999-02-01',
       '1999-12-01', '2000-01-01', '2000-02-01', '2000-12-01',
       '2001-01-01', '2001-02-01', '2001-12-01', '2002-01-01',
       '2002-02-01', '2002-12-01', '2003-01-01', '2003-02-01',
       '2003-12-01', '2004-01-01', '2004-02-01', '2004-12-01',
       '2005-01-01', '2005-02-01', '2006-01-01', '2006-

In [8]:
NDSI_SON_beforSL

<xarray.Dataset>
Dimensions:      (longitude: 14234, latitude: 6200, z: 105)
Coordinates:
  * longitude    (longitude) float64 5.7 5.7 5.701 5.701 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) object '1984-12-01' '1985-01-01' ... '2021-12-01'
Data variables:
    DEM          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    snow_final   (z, latitude, longitude) float32 nan nan nan ... nan nan nan
    snow_final2  (z, latitude, longitude) float32 nan nan nan ... nan nan nan
    crs          (longitude) int32 -2147483647 -2147483647 ... -2147483647
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-02-08 10:38:21
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

####j'attribue un CRS à mon dataset
crs = CRS.from_string('EPSG:4326')
summer_mask.attrs["crs"] = crs.to_proj4()

In [9]:
lat = NDSI_SON_beforSL['latitude'].values
lon = NDSI_SON_beforSL['longitude'].values

summer_mask = xr.DataArray(summer_mask.layer.values, coords={'latitude': lat,'longitude':lon}, dims=['latitude','longitude'])
NDSI_SON_beforSL['Summer_mask'] = summer_mask
####on change en entier le DEM on a pas besoisn de virgule
#NDSI_SON['DEM'] = NDSI_SON.DEM.astype('int16')

In [16]:
NDSI_SON_beforSL

<xarray.Dataset>
Dimensions:      (longitude: 14234, latitude: 6200, z: 105)
Coordinates:
  * longitude    (longitude) float64 5.7 5.7 5.701 5.701 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) object '1984-12-01' '1985-01-01' ... '2021-12-01'
Data variables:
    DEM          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    snow_final   (z, latitude, longitude) float32 nan nan nan ... nan nan nan
    snow_final2  (z, latitude, longitude) float32 nan nan nan ... nan nan nan
    crs          (longitude) int32 -2147483647 -2147483647 ... -2147483647
    Summer_mask  (latitude, longitude) float32 nan nan nan nan ... nan nan nan
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-02-08 10:38:21
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

In [10]:
del summer_mask

In [11]:
##### on fait le summer mas, si summer mask =1 et dans notre ratser c'est unnuage alors on le change en 1
#### sinon on garde les valeurs du dataset

NDSI_SON_beforSL['snow_final3'] = NDSI_SON_beforSL['snow_final2'].copy()
for i in range(len(NDSI_SON_beforSL.z)):
    print(i)
    dataset = NDSI_SON_beforSL.isel(z=i)
    NDSI_SON_beforSL['snow_final3'].values[i] = xr.where((dataset['Summer_mask'] == 1) & (dataset['snow_final2'] == 2), 100, dataset['snow_final2'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


In [29]:
print(np.sum(NDSI_SON_beforSL.snow_final3.isel(z=47) == 100))

<xarray.DataArray 'snow_final3' ()>
array(9369)
Coordinates:
    z        <U10 '2000-12-01'


In [12]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_cloudpercent_b(array):
    cloud = np.sum(array == 2)
    #print(cloud)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    percentage_cloud_b = round(((float(cloud) / float(tot_pix)) * 100),2)
    return percentage_cloud_b

In [13]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_cloudpercent_a(array_copy):
    cloud = np.sum(array_copy  == 2)
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    cloud_percent = round((float(cloud) / float(tot_pix)) * 100, 2)
    return cloud_percent

In [14]:
def step2_gapfilling_tot(array,array_copy):
    cloudpercent_b = compute_cloudpercent_b(array)
    cloudpercent_a = compute_cloudpercent_a(array_copy)
    step2_gf_tot = round(((cloudpercent_b)-(cloudpercent_a)),2)
    #step3_gf_tot = ((cloudpercent_b)-(cloudpercent_a))
    return step2_gf_tot

In [15]:
def compute_snowpercent_b(array):
    snow = np.count_nonzero((array == 1) | (array == 100))
    #print(snow)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    snow_percent_b = round(((float(snow) / float(tot_pix)) * 100),2)
    return snow_percent_b

In [16]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_snowpercent_a(array_copy):
    snow = np.count_nonzero((array_copy == 1) | (array_copy == 100))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    snow_percent = round((float(snow) / float(tot_pix)) * 100, 2)
    return snow_percent

In [17]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_landpercent_b(array):
    land = np.count_nonzero((array == 0) | (array == 200))
    #print(land)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    land_percent_b = round((float(land) / float(tot_pix)) * 100,2)
    return land_percent_b

In [18]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_landpercent_a(array_copy):
    land = np.count_nonzero((array_copy == 0) | (array_copy == 200))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    land_percent = round((float(land) / float(tot_pix)) * 100, 2)
    return land_percent

In [19]:
cloud_percent_b_list = []
cloud_percent_a_list = []
step2_gapfilling_tot_list = []
snow_percent_b_list = []
snow_percent_a_list = []
land_percent_b_list = []
land_percent_a_list = []
time_list = []

In [20]:
for i in range(len(NDSI_SON_beforSL.z)):
    print(i)
    cloud_percent_b = compute_cloudpercent_b(NDSI_SON_beforSL.snow_final2.isel(z=i))
    cloud_percent_b_list.append(cloud_percent_b)

    snow_percent_b = compute_snowpercent_b(NDSI_SON_beforSL.snow_final2.isel(z=i))
    snow_percent_b_list.append(snow_percent_b)
    land_percent_b = compute_landpercent_b(NDSI_SON_beforSL.snow_final2.isel(z=i))
    land_percent_b_list.append(land_percent_b)

    time = pd.to_datetime(NDSI_SON_beforSL.z.isel(z=i).values)
    formatted_date = time.strftime('%Y-%m-%d')
    time_list.append(formatted_date)


    cloud_percent_a = compute_cloudpercent_a(NDSI_SON_beforSL.snow_final3.isel(z=i))
    cloud_percent_a_list.append(cloud_percent_a)

    snow_percent_a = compute_snowpercent_a(NDSI_SON_beforSL.snow_final3.isel(z=i))
    snow_percent_a_list.append(snow_percent_a)
    land_percent_a = compute_landpercent_a(NDSI_SON_beforSL.snow_final3.isel(z=i))
    land_percent_a_list.append(land_percent_a)

    step2_gf_tot = step2_gapfilling_tot(NDSI_SON_beforSL.snow_final2.isel(z=i),NDSI_SON_beforSL.snow_final3.isel(z=i))
    step2_gapfilling_tot_list.append(step2_gf_tot)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


In [21]:
data = {
    'Time': time_list,
    'Cloud_Percent_Before': cloud_percent_b_list,
    'Cloud_Percent_After': cloud_percent_a_list,
    'Cloud_Reduction': step2_gapfilling_tot_list,
    'Snow_Percent_Before': snow_percent_b_list,
    'Snow_Percent_After': snow_percent_a_list,
    'Land_Percent_Before': land_percent_b_list,
    'Land_Percent_After': land_percent_a_list
}

# Créer un DataFrame à partir du dictionnaire
df_step2 = pd.DataFrame(data)
filename = f'df_summer_mask_DJF.xlsx'
filepath = os.path.join('/home/charlottepoussin/Excels', filename)
df_step2.to_excel(filepath, index=False)

In [22]:
df_step2

,Time,Cloud_Percent_Before,Cloud_Percent_After,Cloud_Reduction,Snow_Percent_Before,Snow_Percent_After,Land_Percent_Before,Land_Percent_After
0,1984-12-01,54.37,54.31,0.06,35.21,35.27,10.42,10.42
1,1985-01-01,45.10,44.95,0.15,46.40,46.55,8.50,8.50
2,1985-02-01,42.76,42.56,0.20,55.91,56.10,1.33,1.33
3,1985-12-01,51.23,51.07,0.16,37.39,37.54,11.38,11.38
4,1986-01-01,54.13,53.96,0.17,38.07,38.25,7.80,7.80
...,...,...,...,...,...,...,...,...
100,2020-02-01,5.92,5.92,0.00,52.20,52.20,41.88,41.88
101,2020-12-01,43.90,43.89,0.01,44.39,44.40,11.71,11.71
102,2021-01-01,32.55,32.54,0.01,59.20,59.21,8.25,8.25
103,2021-02-01,2.79,2.79,0.00,60.29,60.29,36.92,36.92


In [28]:
NDSI_SON_beforSL

<xarray.Dataset>
Dimensions:      (longitude: 14234, latitude: 6200, z: 105)
Coordinates:
  * longitude    (longitude) float64 5.7 5.7 5.701 5.701 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) object '1984-12-01' '1985-01-01' ... '2021-12-01'
Data variables:
    DEM          (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    snow_final   (z, latitude, longitude) float32 nan nan nan ... nan nan nan
    snow_final2  (z, latitude, longitude) float32 nan nan nan ... nan nan nan
    crs          (longitude) int32 -2147483647 -2147483647 ... -2147483647
    Summer_mask  (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    snow_final3  (z, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-02-08 10:38:21
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

In [ ]:
print(np.sum(NDSI_SON_beforSL.snow_final3.isel(z=47) == 100))

In [30]:
NDSI_DJF_a_12_2000 = NDSI_SON_beforSL.snow_final3.isel(z=47)

In [35]:
print(np.sum(NDSI_DJF_a_12_2000 == 100))

<xarray.DataArray 'snow_final3' ()>
array(9369)
Coordinates:
    z        <U10 '2000-12-01'


In [ ]:
NDSI_SON_beforSL.to_netcdf('NDSI_DJF_summermask_nc.nc')

In [34]:
NDSI_DJF_a_12_2000.to_netcdf('step3_2000_12_nc.nc')